In [1]:
# for hide warning messages
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# import
from Pthougt_encode import Pthought_vec
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity as cs

## Glove path and Load data 

In [3]:
glove_path = "D:/glove.840B.300d.txt"
word_emb_dim = 300

set_tst = pd.read_csv("./data/2017val.csv")
cap = np.array(set_tst['caption'])
img_id = set_tst['image_id']
uniq_id = np.unique(img_id)

## Build vocabulary 

In [4]:
sess = tf.Session()
model = Pthought_vec(1200,2,300,bi_direction=False)
model.set_glove_path(glove_path)
model.build_vocab(cap,tokenize = True)

placeholders created !
encoder created !
Found 8414(/8641) words with glove vectors
Vocab size : 8414


## Load encoder (two-layer Forward RNN encoder)

In [5]:
model.enc_saver()
model.load_enc(model_path= './model/two_forward',model_name="Pthought_net.ckpt",sess=sess)

INFO:tensorflow:Restoring parameters from ./model/two_forward/Pthought_net.ckpt


## Generate sentence vector 

In [6]:
total = model.encode(sess, cap, tokenize = True)

## Calculate P-coherence 

In [7]:
coherence = []
for i in range(len(uniq_id)):
    # inner
    tmp_id = np.where(img_id == uniq_id[i])[0]
    coherence_vec = np.take(total, tmp_id, 0)
    dist_coherence = cs(coherence_vec)
    mean_coherence = (np.sum(dist_coherence) - len(dist_coherence)) / (len(dist_coherence) * len(dist_coherence) - len(dist_coherence))
    coherence.append(mean_coherence)

In [8]:
print("P-coherence : %.3f" %(np.mean(coherence)))

P-coherence : 0.790
